In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,714 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:13 https://

In [4]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Hashing").getOrCreate()

In [5]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [6]:
# Sample DataFrame with repeating words
dataframe = spark.createDataFrame([
    (0, "The cow cow jumped and jumped cow"),
    (1, "then the cow said"),
    (2, "I am a cow that jumped")
],["id", "words"])

dataframe.show()

+---+--------------------+
| id|               words|
+---+--------------------+
|  0|The cow cow jumpe...|
|  1|   then the cow said|
|  2|I am a cow that j...|
+---+--------------------+



In [7]:
# Tokenize the words
tokenizer = Tokenizer(inputCol="words", outputCol="tokens")
wordsData = tokenizer.transform(dataframe)
wordsData.show()

+---+--------------------+--------------------+
| id|               words|              tokens|
+---+--------------------+--------------------+
|  0|The cow cow jumpe...|[the, cow, cow, j...|
|  1|   then the cow said|[then, the, cow, ...|
|  2|I am a cow that j...|[i, am, a, cow, t...|
+---+--------------------+--------------------+



In [8]:
# Run the hashing term frequency
hashing = HashingTF(inputCol="tokens", outputCol="hashedValues", numFeatures=pow(2,4))

# Transform into a DF
hashed_df = hashing.transform(wordsData)

In [9]:
# Display new DataFrame
hashed_df.show(truncate=False)

+---+---------------------------------+-----------------------------------------+----------------------------------------+
|id |words                            |tokens                                   |hashedValues                            |
+---+---------------------------------+-----------------------------------------+----------------------------------------+
|0  |The cow cow jumped and jumped cow|[the, cow, cow, jumped, and, jumped, cow]|(16,[1,3,6,11],[1.0,3.0,2.0,1.0])       |
|1  |then the cow said                |[then, the, cow, said]                   |(16,[0,1,3,13],[1.0,1.0,1.0,1.0])       |
|2  |I am a cow that jumped           |[i, am, a, cow, that, jumped]            |(16,[0,3,6,12,13],[1.0,2.0,1.0,1.0,1.0])|
+---+---------------------------------+-----------------------------------------+----------------------------------------+



In [10]:
# Fit the IDF on the data set 
idf = IDF(inputCol="hashedValues", outputCol="features")
idfModel = idf.fit(hashed_df)
rescaledData = idfModel.transform(hashed_df)

In [2]:
# Display the DataFrame
rescaledData.select("tokens", "features").show(truncate=False)

NameError: ignored